In [ ]:
import numpy as np
from scipy.stats import norm
from matplotlib import pyplot as plt
import seaborn as sns
import yaml
import os
import json
sns.set_context('paper', font_scale =1.5)

In [ ]:
# relevant functions for the sequences
def adjacency_check(shape_sequence: np.array,shapes_no_target:list):
    """
    Checks whether any identical shapes are placed adjacently in a sequence and corrects for it

    Args:
        shape_sequence (np.array): shape sequence with no targets
        shapes_no_target (list): list of shapes without the target

    Returns:
        array: shape sequence fulfilling the adjacency rule
    """
    adj_count = 0
    for i in range(len(shape_sequence) - 1):
        if shape_sequence[i] == shape_sequence[i + 1]:
            adj_count += 1
        # Find a different letter to replace the adjacent duplicate
            for shape in shapes_no_target: #replacing with any other letter other than the target letter
                if shape != shape_sequence[i] and (i == 0 or shape != shape_sequence[i - 1]):
                    shape_sequence[i + 1] = shape
                    break
             
    return shape_sequence.tolist()


def place_targets_in_sequence(init_shape_sequence: list, target_shape: str,inter_targ_dist: int, max_target_dist: int, ind_to_avoid: list = None):
    """
    Places the target shape in the input sequence 

    Args:
        init_shape_sequence (np.array): initial shape sequence without targets
        target_shape (str): chosen target shape
        inter_targ_dist (int): minimum distance between two targets (in indices)
        max_target_dist (int): maximum possible distance between targets
        ind_to_avoid (list, optional): indices to be avoided when placing the targets. Used for non-cued sequences. Defaults to None.

    Returns:
        array: input sequence with targets at random positions
    """
    
    if ind_to_avoid is None:
        current_index = 0    
        while current_index < len(init_shape_sequence):
        
            if current_index == 0:
                # for the first target, we utilize the initial indices of the sequence
                tar_dist = np.random.choice(np.arange(2, len(init_shape_sequence) // 10))

            else:
                tar_dist = np.random.choice(np.arange(inter_targ_dist, max_target_dist))  # tar_dist now varies from inter_targ_dist to a max value

            # Ensure current_index + tar_dist does not exceed sequence_size
            if current_index + tar_dist < len(init_shape_sequence):
                init_shape_sequence[current_index + tar_dist] = target_shape
                current_index += tar_dist
            else:
                break
    else:
        # targets_placed = 0
        current_index = 0           

        while current_index < len(init_shape_sequence):
        
            if current_index == 0:
                # for the first target, we utilize the initial indices of the sequence
                tar_dist = np.random.choice(np.arange(2, len(init_shape_sequence) // 10))

            else:
                tar_dist = np.random.choice(np.arange(inter_targ_dist, max_target_dist))  # tar_dist now varies from inter_targ_dist to a max value

            # Ensure current_index + tar_dist is not equal to any of the target indices in ind_to_avoid
            # If true, skips to next index and tries again 
            ind_avoid_condition = np.where(ind_to_avoid == (current_index + tar_dist))[0].size
            
            if ind_avoid_condition == 0:
                
                # Ensure current_index + tar_dist does not exceed sequence_size
                if (current_index + tar_dist) < len(init_shape_sequence):                    
                    init_shape_sequence[current_index + tar_dist] = target_shape
                    current_index += tar_dist
                    # targets_placed += 1

                else:
                    break
            else:
                current_index += 1
     
    shape_sequence_with_targets = init_shape_sequence

    return shape_sequence_with_targets

def matching_indices(a: list,b: list):
    """
    Checks if any indices of the two lists have matching values. This is done to avoid the simultaneous presentation of one shape on both the cued and non_cued sides.
    
    Args: 
        a,b (list) : cued and non_cued sequences
    Returns:
        int: count of matching indices
        list: indices for which values in both the input lists

        
    """
    ind = []
    if type(a) != list:
        a = a.tolist()
    
    if type(b) != list:
        b = b.tolist()

    match = 0
    
    for i in range(len(a)):
        if a[i] == b[i]:
            ind.append(i)
            match += 1

    return match, ind


def gen_cued_non_cued_seq(init_shape_sequence: list, target_shape: str, inter_targ_dist: int, max_target_dist: int, shapes_no_target: list, mapping_dict: dict):
    """
    Generates the cued and the non_cued sequences

    Args:
        init_shape_sequence (np.array): initial shape sequence without targets
        target_shape (str): chosen target shape
        inter_targ_dist (int): minimum distance between two targets (in indices)
        max_target_dist (int): maximum possible distance between targets
        shapes_no_target (list): list of shapes without the target

    Returns:
        array: cued and non_cued sequences with targets
    """
    
     # adjacency check
    cued_seq_no_targ = adjacency_check(shape_sequence=init_shape_sequence, shapes_no_target=shapes_no_target)

    # non_cued sequence (without targets), satisfies adjacency rule implicitly
    non_cued_seq_no_targ = [mapping_dict[shape] for shape in cued_seq_no_targ]

    # placing targets in cued sequence 
    cued_seq_targ = place_targets_in_sequence(cued_seq_no_targ, target_shape, inter_targ_dist, max_target_dist)

    # collecting indices at which targets were placed in the cued_sequence   
    targ_ind_cued = [ind for ind, shape in enumerate(cued_seq_targ) if shape == target_shape]   
    
    # placing targets in non_cued_sequence
    non_cued_seq_targ = place_targets_in_sequence(non_cued_seq_no_targ, target_shape, inter_targ_dist, max_target_dist, ind_to_avoid= targ_ind_cued)
    

    return cued_seq_targ, non_cued_seq_targ

   


In [ ]:
# relevant function for visualization of sequence parameters: std deviation of target counts, and target count distributions
def visualize_seq_params(d_max_range: np.array, std_all_sequences: np.array, all_target_counts: np.array , sim_size:int, seq_type: str, std_plot=False,dist_plot=False):
    """
    Plots the standard deviation of target counts and the target count distribution for all simulations 

    Args:
        max_possible_dist (int): maximum possible distance between two targets
        d_max_range (np.array): range of maximum distances between targets, starts from 10 and goes to max_possible_dist
        std_all_sequences (np.array): standard deviation of target counts (for cued or non_cued)
        all_target_counts (np.array): number of targets in sequences across all simulations
        sim_size (int): number of simulations
        seq_type (str): type of sequence (cued or non_cued)
        std_plot (bool): plot the variation of standard deviation of target counts with d_max_range
        dist_plot (bool): plot the fitted gaussian and the histogram of the corresponding target count distribution 

    """
    
    # plotting variation of standard deviation
    if std_plot:
        plt.figure(figsize = (11.69, 4))
        plt.plot(d_max_range, std_all_sequences, '-*')
        plt.title(f"std for all maximum distances, max std at d_max = {d_max_range[np.argmax(std_all_sequences)]}")
        plt.xlabel('d_max')
        plt.ylabel('standard deviation')
        plt.ylim([0, np.max(std_all_sequences)+ 0.05])
        plt.axhline(np.max(std_all_sequences), color = 'r')
        plt.axvline(d_max_range[np.argmax(std_all_sequences)], color = 'r')


    # plotting count distribution at maximum standard deviation    
    data = all_target_counts[np.argmax(std_all_sequences)]
    mu, std = norm.fit(data)
    if dist_plot:        
        plt.figure(figsize = (11.69, 4))
        plt.hist(data, bins=25, density=True, alpha=0.2, color='green')
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 100)
        p = norm.pdf(x, mu, std)
        plt.plot(x, p, color='green', linewidth=3)
        plt.title(f"{seq_type}: target count distribution for {sim_size} simulations at max std")
        plt.xlabel('number of targets')
        plt.ylabel('target frequency')
        plt.show()
    


## Explanation: 
    1. We are generating sequences where targets can be placed at a maximum of D spaces apart. 

    2. This maximum possible distance between targets is varied, and for each variation, we generate a total of N sequences

    3. We then find the target count for each of the D x N sequences, and compute a distribution of this count. 

    4. The optimum value for the maximum target distance, d_max, corresponding to the maximum standarad deviation of these count distributions is chosen as the final parameter to generate the sequences used in our experiment.

    5. The 1-1 mapping [1] for sequences still has to be maintained, i.e., cued and non_cued sequences will have a common d_max value such that they come from the same subset.

**[1]**: The sequence sets (cued and non cued) are 1-1 mapped. This means that for an ith cued sequence, the conditions of adjacency, simultaneity (see matching_indices function) and target placement are met ONLY for the corresponding ith non_cued sequence.

**Note**: This notebook will use the terms "cued" and "non_cued" sequences throughout. The "cued" sequences are the shape sequences that the subject will covertly/overtly pay attention to during a trial, and the "non_cued" sequences are the sequences that are not attended to. This terminology is limited to an offline protocol. In an online protocol, there will be no such difference as the cued and non_cued sequences. Target balancing (see next explanation section) will then be done for both categories of sequences.

In [ ]:
# Parameters
project_path = r'C:\Users\s1081686\Desktop\RA_Project\Scripts\pynt_codes\SN_experiment\experiment_version_2'
with open(os.path.join(project_path,'config.yml'), "r") as yaml_file:
    config_data = yaml.safe_load(yaml_file)
    
# load constants
experiment_params = config_data['experimental_params']
SHAPES = experiment_params['SHAPES']
FR = experiment_params['FR']
TRIAL_TIME = experiment_params['TRIAL_TIME']

# compute parameters
shape_change_time_sec = .250 # The duration (sec) between the occurrance of two different shapes

total_frames = int(TRIAL_TIME*FR) # Total frames in a trial    

change_shapes = int(shape_change_time_sec*FR)  # The number of frames after which a new shape will appear inside the circle

sequence_size = int(np.ceil(total_frames /change_shapes)) # size of sequence

target_shape = SHAPES[-1]

inter_targ_dist = 6 # the difference in indices between two target positions

# Initialize sequence
shapes_no_target = [shape for shape in SHAPES if shape != target_shape]

init_shape_sequence = np.random.choice(shapes_no_target, size=sequence_size).tolist()

mapping_dict = {shapes_no_target[0]:shapes_no_target[3], 
                shapes_no_target[1]:shapes_no_target[2], 
                shapes_no_target[2]:shapes_no_target[1], 
                shapes_no_target[3]:shapes_no_target[0],
                }


# Simulation param or 'N'
sim_size = 5000

# Maximum distance between targets param
D = 40
d_max_range = np.arange(10, D +1, 3)
print("max possiblle distance between targets:", max(d_max_range))

# sequence params
all_sequences_cued = np.zeros((len(d_max_range), sim_size, sequence_size), dtype = 'object')
all_sequences_non_cued = np.zeros((len(d_max_range), sim_size, sequence_size), dtype = 'object')

# target count params
all_target_counts_cued = np.zeros((len(d_max_range), sim_size)) 
all_target_counts_non_cued = np.zeros((len(d_max_range), sim_size)) 



# generating D x N sequences
for i_max_dist, max_dist in enumerate(d_max_range):

    i_sim = 0
    
    while i_sim != sim_size:

        init_shape_sequence = np.random.choice(shapes_no_target, size=sequence_size)
        
        # generating cued and non_cued sequences
        cued_seq_targ, non_cued_seq_targ = gen_cued_non_cued_seq(init_shape_sequence = init_shape_sequence, 
                                                                target_shape = target_shape, 
                                                                inter_targ_dist = inter_targ_dist, 
                                                                max_target_dist = max_dist, 
                                                                shapes_no_target = shapes_no_target, 
                                                                mapping_dict = mapping_dict)
            
        # checking if the generated sequence meets our criteria (the difference between all target indices should be atleast the minimum distance)

        # cued
        target_indices_cued = [index for index, shape in enumerate(cued_seq_targ) if shape == target_shape]
        diff_cued = np.diff(target_indices_cued)
        
        # non_cued
        target_indices_non_cued = [index for index, shape in enumerate(non_cued_seq_targ) if shape == target_shape]
        diff_non_cued = np.diff(target_indices_non_cued)
        
        print(f"checking for target condition, i_sim = {i_sim} ")
        if len(target_indices_cued) != len(target_indices_non_cued):
            
            print(len(target_indices_cued),len(target_indices_non_cued))
            
            if np.all(diff_cued >= inter_targ_dist) and np.all(diff_non_cued >= inter_targ_dist):
                all_sequences_cued[i_max_dist, i_sim] = cued_seq_targ
                all_target_counts_cued[i_max_dist, i_sim] = len(target_indices_cued) # number of targets in the sequence
                
                all_sequences_non_cued[i_max_dist, i_sim] = non_cued_seq_targ
                all_target_counts_non_cued[i_max_dist, i_sim] = len(target_indices_non_cued) # number of targets in the sequence
            i_sim += 1
            print(f"num targets unqequal for both sides in a trial, proceeding, i_sim:{i_sim}")
        else:
            print(f"num targets equal, trying again. i_sim: {i_sim}")
            

print("shape of all sequences",all_sequences_cued.shape)
print("target count for all simulations", all_target_counts_cued.shape)

In [ ]:
# Computing standard deviation of target counts both sequences and then averaging them.
# We need to pick the same d_max for both sets of sequences to retain their 1-1 mapping

std_all_sequences_cued = np.zeros((len(d_max_range),))
std_all_sequences_non_cued = np.zeros((len(d_max_range),))

for i in range(len(d_max_range)):
    # cued
    data_cued = all_target_counts_cued[i]
    _, std_cued = norm.fit(data_cued)
    std_all_sequences_cued[i] =  std_cued
    
    # non_cued
    data_non_cued = all_target_counts_non_cued[i]
    _, std_non_cued = norm.fit(data_non_cued)
    std_all_sequences_non_cued[i] =  std_non_cued

    
std_both_sequences = (std_all_sequences_cued + std_all_sequences_non_cued) / 2

# plotting variation of standard deviation and target count distribution
visualize_seq_params(d_max_range=d_max_range, 
                     std_all_sequences=std_both_sequences,
                     all_target_counts=all_target_counts_cued ,
                     sim_size = sim_size, 
                     seq_type = 'sequence set A',
                     std_plot=True,
                     dist_plot= True)

visualize_seq_params(d_max_range=d_max_range, 
                     std_all_sequences=std_both_sequences, 
                     all_target_counts=all_target_counts_non_cued, 
                     sim_size = sim_size, 
                     seq_type = 'sequence set B',
                     std_plot=False,
                     dist_plot= True)


#### Balancing target counts for left and right cued_sequences 

    1.The p300 response ellicited by the shape sequences need to be balanced for both cued sides

    2.This is done by randomly selecting two unique sets of indices such that the sum of target counts found using these indices are be balanced within both cued and non cued indices. 

**Note**: The splitting of cued sequences to the left or right side is only relevant for balancing. Once this is done, they can be arbritrarily used for either of the sides irrespective of which sequence set (right or left) they initially belonged to. The non_cued sequences also undergo target balancing as this would be necessary in an online setting.


In [ ]:
def balance_target_counts(total_sequences, target_count_non_cued, target_count_cued, num_seq_all_runs):
    """
    Balances target counts for the left and right cued sequences for a participant (sum of target counts are equal when considering all runs)
    
    Args:
        total_sequences (int): number of total cued/non_cued sequences with the maximum std in their target counts
        target_count_cued (np.array): corresponding target counts of the cued sequences with max std
        target_count_non_cued (np.array): corresponding target counts of the non_cued sequences with max std
        num_seq_all_runs (int): number of sequences for all runs of an experimental condition (covert or overt)

    Returns:
        array: set of indices for which targets are balanced for both cued and non_cued sequences
    """
    # define to sets of indices (each with length 50% of the total sequences required in all runs)
    ind_s1 = np.random.choice(total_sequences, size=num_seq_all_runs // 2, replace=False) 
    ind_s2 = np.random.choice(total_sequences, size=num_seq_all_runs // 2, replace=False)
    
    # no indices should match in the two sets (we dont want sequences to be repeated)
    while np.any(np.in1d(ind_s1, ind_s2)):
        ind_s2 = np.random.choice(total_sequences, size=num_seq_all_runs // 2, replace=False)    

    # sum of target counts: cued sequences (sequence set A)
    targ_sum_cued_s1 = target_count_cued[ind_s1].sum()
    targ_sum_cued_s2 = target_count_cued[ind_s2].sum()
    
    # sum of target counts: non cued sequences (sequence set B)
    targ_sum_non_cued_s1 = target_count_non_cued[ind_s1].sum()
    targ_sum_non_cued_s2 = target_count_non_cued[ind_s2].sum()
    
    # cond_1 = targ_sum_cued_s1 == targ_sum_cued_s2
    # cond_2 = targ_sum_non_cued_s1 == targ_sum_non_cued_s2
    
    cond_1 = np.abs(targ_sum_cued_s1 - targ_sum_cued_s2) < 5
    cond_2 = np.abs(targ_sum_non_cued_s1 == targ_sum_non_cued_s2) < 5

    
    while ~(cond_1 and cond_2):
        
        # (re)define to sets of indices 
        ind_s1 = np.random.choice(total_sequences, size=num_seq_all_runs // 2, replace=False) 
        ind_s2 = np.random.choice(total_sequences, size =num_seq_all_runs // 2, replace=False)
        
        # no indices should match in the two sets
        while np.any(np.in1d(ind_s1, ind_s2)):
            ind_s2 = np.random.choice(total_sequences, size=num_seq_all_runs // 2, replace=False)     
        
        # compute sum of target counts again
        targ_sum_cued_s1 = target_count_cued[ind_s1].sum()
        targ_sum_cued_s2 = target_count_cued[ind_s2].sum()
        
        targ_sum_non_cued_s1 = target_count_non_cued[ind_s1].sum()
        targ_sum_non_cued_s2 = target_count_non_cued[ind_s2].sum()
        
        cond_1 = np.abs(targ_sum_cued_s1 - targ_sum_cued_s2) < 5
        cond_2 = np.abs(targ_sum_non_cued_s1 - targ_sum_non_cued_s2) < 5
        
        # cond_1 = targ_sum_cued_s1 == targ_sum_cued_s2
        # cond_2 = targ_sum_non_cued_s1 == targ_sum_non_cued_s2  
        print("cued",targ_sum_cued_s1,targ_sum_cued_s2)
        print("non cued",targ_sum_non_cued_s1,targ_sum_non_cued_s2)
        
    print("exiting while loop")  
    print("cued",targ_sum_cued_s1,targ_sum_cued_s2)
    print("non cued",targ_sum_non_cued_s1,targ_sum_non_cued_s2)
    return ind_s1, ind_s2

#### Storing sequence and target info for P participants

In [ ]:
# 1. selecting the sequences with maximum standard deviation
cued_seq_max_sd = all_sequences_cued[np.argmax(std_both_sequences)]
non_cued_seq_max_sd = all_sequences_non_cued[np.argmax(std_both_sequences)]

# 2. number of targets in the corresponding sequences with maximum standard deviation
target_counts_cued_max_sd = all_target_counts_cued[np.argmax(std_both_sequences)]

# 3. targets in the corresponding non_cued_sequences
target_counts_non_cued_max_sd = all_target_counts_non_cued[np.argmax(std_both_sequences)]


# Parameters for the experiment 
P = 40 # number of participants
n_runs_cov = 4 # total number of blocks/ runs (4 covert + 1 overt)
n_runs_ov = 1
n_trials = 20 # total trials

# Number of cued and non_cued sequences in an experiment = n_runs x n_trials

# number of sequences: covert (cov)
total_seq_cued_cov = n_runs_cov * n_trials
total_seq_non_cued_cov = n_runs_cov * n_trials

# number of sequences: overt (ov)
total_seq_cued_ov = n_runs_ov * n_trials
total_seq_non_cued_ov = n_runs_ov * n_trials

# looping over participants and storing sequence information for both overt (no balancing needed) and covert conditions
for i_par in range(P):
    
    
    #overt: cued sequences (no balancing needed, but could be added easily. see covert sections below)
    cued_indices_ov = np.random.choice(cued_seq_max_sd.shape[0], size = total_seq_cued_ov, replace = False) # sequence indices

    cued_seq_ov = cued_seq_max_sd[cued_indices_ov]# sequences
    cued_targ_counts_ov = target_counts_cued_max_sd[cued_indices_ov]# target counts
    
    # overt: non_cued sequences
    non_cued_indices_ov = cued_indices_ov # sequence indices
    
    non_cued_seq_ov = non_cued_seq_max_sd[non_cued_indices_ov] # sequences
    non_cued_targ_counts_ov = target_counts_non_cued_max_sd[non_cued_indices_ov] # target counts
    
    
    # covert: get indices such that target counts are balanced in cued and non_cued sequences
    ind_s1, ind_s2 = balance_target_counts(total_sequences=cued_seq_max_sd.shape[0], 
                                        target_count_cued=target_counts_cued_max_sd,
                                        target_count_non_cued=target_counts_non_cued_max_sd,
                                        num_seq_all_runs=total_seq_non_cued_cov)
    balanced_indices = np.concatenate((ind_s1, ind_s2))
    # get cued, non_cued sequences and their corresponding target counts
    
    cued_seq_cov, cued_targ_counts_cov = cued_seq_max_sd[balanced_indices], target_counts_cued_max_sd[balanced_indices]
    non_cued_seq_cov, non_cued_targ_counts_cov = non_cued_seq_max_sd[balanced_indices], target_counts_non_cued_max_sd[balanced_indices]
    
    # checking if within a trial sequences selected have unequal trials  
    print('*'*100)
    print(f"performing validation tests on target counts and sequences within a trial and across runs for P{i_par + 1}")  
    
    count_A = 0
    count_B = 0
    
    for i in range(cued_targ_counts_cov.shape[0]):

        if cued_targ_counts_cov[i] == non_cued_targ_counts_cov[i]:
            count_A += 1
        else:
            count_B += 1
    if count_A > 0 :
        print("failure: target conditions within a trial were not met")
    else:
        print(f"success: target conditions within a trial were met")
    
    # checking target conditions across the run
    if np.abs(target_counts_cued_max_sd[ind_s1].sum() - target_counts_cued_max_sd[ind_s2].sum()) < 5:
        print("success: covert cued: target conditions across the run were met")
        
    else:
        print("failure: covert cued: target conditions across the run were not met")
        print(f"difference is {np.abs(target_counts_cued_max_sd[ind_s1].sum() - target_counts_cued_max_sd[ind_s2].sum())}")
    
    # checking target conditions across the run
    if np.abs(target_counts_non_cued_max_sd[ind_s1].sum() - target_counts_non_cued_max_sd[ind_s2].sum()) < 5:
        print("success: covert non_cued: target conditions across the run were met")
        
    else:
        print("covert: covert non_cued:covert target conditions across the run were not met")
        print(f"difference is {np.abs(target_counts_non_cued_max_sd[ind_s1].sum() - target_counts_non_cued_max_sd[ind_s2].sum())}") 
        
        break  
    
        
        
    # checking if any of the shapes in the cued and non-cued sequences are identical for the same index
    match_cov = 0
    for i in range(cued_seq_cov.shape[0]):
        match, _ = matching_indices(cued_seq_cov[i], non_cued_seq_cov[i])
        match_cov += match
        
    if match_cov > 0:
        print("failure:covert: shapes are identical for the same indices, sequence condition not met")
        break
    else:
        print("success: covert: no matching shapes for the same index sequence")
        
    match_ov = 0
    for i in range(cued_seq_ov.shape[0]):
        match, _ = matching_indices(cued_seq_ov[i], non_cued_seq_ov[i])
        match_cov += match
        
    if match_cov > 0:
        print("failure:overt: shapes are identical for the same indices, sequence condition not met")
        break
    else:
        print("success:overt: no matching shapes for the same index sequence")
    print('*'*100)
    # collecting info for participant P: overt
    P_seq_info_ov = {'cued_sequences':cued_seq_ov.tolist(), 'non_cued_sequences': non_cued_seq_ov.tolist()}
    P_target_count_info_ov = {'cued_target_counts': cued_targ_counts_ov.tolist(), 'non_cued_target_counts': non_cued_targ_counts_ov.tolist()}
    P_all_info_ov = {'sequence_info': P_seq_info_ov, 'target_count_info': P_target_count_info_ov}
    
    # collecting info for participant P: covert
    P_seq_info_cov = {'cued_sequences':cued_seq_cov.tolist(), 'non_cued_sequences': non_cued_seq_cov.tolist()}
    P_target_count_info_cov = {'cued_target_counts': cued_targ_counts_cov.tolist(), 'non_cued_target_counts': non_cued_targ_counts_cov.tolist()}
    P_all_info_cov = {'sequence_info': P_seq_info_cov, 'target_count_info': P_target_count_info_cov}
    

  
    # # saving
    # fn_ov = rf"C:\Users\s1081686\Desktop\RA_Project\Scripts\pynt_codes\SN_experiment\experiment_version_2\shape_sequences\P{i_par+1}_overt.json"
    # with open(fn_ov, 'w') as f: 
    #     json.dump(P_all_info_ov, f)
    
    # fn_cov = rf"C:\Users\s1081686\Desktop\RA_Project\Scripts\pynt_codes\SN_experiment\experiment_version_2\shape_sequences\P{i_par+1}_covert.json"
    # with open(fn_cov, 'w') as f:
    #     json.dump(P_all_info_cov, f)
    

In [ ]:
# sequences check: loading sequences and checking for simultaneity

import json

# Define file paths (assuming you used the file paths from the previous code)
overt_file = rf"C:\Users\s1081686\Desktop\RA_Project\Scripts\pynt_codes\SN_experiment\experiment_version_2\shape_sequences\P{1}_overt.json"
covert_file = rf"C:\Users\s1081686\Desktop\RA_Project\Scripts\pynt_codes\SN_experiment\experiment_version_2\shape_sequences\P{1}_covert.json"

# Load overt data
with open(overt_file, 'r') as f:
  overt_data = json.load(f)

# Load covert data
with open(covert_file, 'r') as f:
  covert_data = json.load(f)

# Access specific information

# Overt sequences and target counts
overt_cued_sequences = np.array(overt_data['sequence_info']['cued_sequences'])
overt_non_cued_sequences = np.array(overt_data['sequence_info']['non_cued_sequences'])
overt_cued_target_counts = np.array(overt_data['target_count_info']['cued_target_counts'])
overt_non_cued_target_counts = np.array(overt_data['target_count_info']['non_cued_target_counts'])

# Covert sequences and target counts (similar structure)
covert_cued_sequences = np.array(covert_data['sequence_info']['cued_sequences'])
covert_non_cued_sequences = np.array(covert_data['sequence_info']['non_cued_sequences'])
covert_cued_target_counts = np.array(covert_data['target_count_info']['cued_target_counts'])
covert_non_cued_target_counts = np.array(covert_data['target_count_info']['non_cued_target_counts'])

match_ov = 0
for i in range(overt_cued_sequences.shape[0]):
  match, _ = matching_indices(overt_cued_sequences[i], overt_non_cued_sequences[i])
  match_ov += match

match_cov = 0
for i in range(covert_cued_sequences.shape[0]):
  match, _ = matching_indices(covert_cued_sequences[i], covert_non_cued_sequences[i])
  match_cov += match

print("covert: indices at which cued and non_cued sequences have indentical shapes", match_ov)
print("covert: indices at which cued and non_cued sequences have indentical shapes", match_cov)